In [ ]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import chromadb


db_path="./vektor_DB"

#lädt datenbank lokal runter
client = chromadb.PersistentClient(path=db_path)

#unser emdedding model
embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

#"meinungen" aus der Datenbank laden
collection = client.get_collection("meinungen")

#retrievet daten von unserer Vektorendatenbank
def query_collection(query_text, n_results=4):

    #encode query text (-> unser input)
    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

In [ ]:
from transformers import pipeline

rag_pipeline = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-0.5B-Instruct",
    device_map="auto",
    torch_dtype="auto" # Use the most efficient data type for the hardware
)


In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb


db_path="./vektor_DB"

#lädt datenbank lokal runter
client = chromadb.PersistentClient(path=db_path)

#unser emdedding model
embedding_model = SentenceTransformer("sentence-transformers/gtr-t5-large")

#"meinungen" aus der Datenbank laden
collection = client.get_collection("meinungen")

#retrievet daten von unserer Vektorendatenbank
def query_collection(query_text, n_results=4):

    #encode query text (-> unser input)
    query_embedding = embedding_model.encode(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    return results

In [ ]:
#Funktion, die die Datenbank abfragt und eine Antwort generiert
def query_with_qwen25(query_text):
    results = query_collection(query_text)
    context = ""
    for i in range(len(results["documents"][0])):
        context += f"Document {i+1}:\n"
        context += f"{results['documents'][0][i]}\n"
        context += f"Party: {results['metadatas'][0][i]['party']}\n"


    prompt = f"""
    You are a political AI assistant.  Based on the following information retrieved from a vector database, answer the user's question.  If the provided data is not relevant to the question, respond with "I do not know".

    User Question: {query_text}

    Retrieved Data:
    {context}

    Your Answer:
    """

    generated_text = rag_pipeline(prompt, max_new_tokens=250)[0]["generated_text"]
    #Extract Anwort von generated text
    answer = generated_text[len(prompt):].strip()
    return answer


query = "Was denkt die AFD über Islamunterricht an deutschen Schulen?"
answer = query_with_qwen25(query)
print(answer)